In [3]:
import os
import glob
import random
import cv2
import obspy
import scipy
import torch

from io import BytesIO
from obspy import Stream
from obspy import read
from obspy.taup import TauPyModel
from obspy.geodetics import gps2dist_azimuth
from obspy.signal.trigger import classic_sta_lta

import numpy as np 
import scipy.signal as signal
import matplotlib.pyplot as plt 

In [2]:
def scale_amplitude(data, rate):
    'Scale amplitude or waveforms'
    
    tmp = np.random.uniform(0, 1)
    if tmp < rate:
        data *= np.random.uniform(1, 3)
    elif tmp < 2 * rate:
        data /= np.random.uniform(1, 3)
    return data

In [3]:
def shift_event(data, maxshift, rate, start, halfdim): 
    'Randomly rotate the array to shift the event location'
    
    if np.random.uniform(0, 1) < rate:
        start += int(np.random.uniform(-maxshift, maxshift))             
    return data[:, start-halfdim:start + halfdim]

In [4]:
def get_wave_arrival(input_Stream):

    # 選取第一個Trace對象
    tr = input_Stream[0]

    # 測站的緯度和經度
    station_latitude = tr.stats.sac.stla
    station_longitude = tr.stats.sac.stlo


    # 地震源的緯度和經度
    event_latitude = tr.stats.sac.evla
    event_longitude = evlo = tr.stats.sac.evlo


    # 使用 ObsPy 的 gps2dist_azimuth 函數來計算距離
    distance_m, azimuth1, azimuth2 = gps2dist_azimuth(station_latitude, station_longitude, event_latitude, event_longitude)
    distance_in_km = distance_m / 1000

    # 將距離從公尺轉換為地震中心距離（以度為單位）
    distance_in_degrees = distance_m / 1000 / 111.195


    # 取得地震源的深度
    depth_in_km = tr.stats.sac.evdp

    # 總距離是地震中心距離與地震深度的和
    total_distance_in_km = distance_in_km + depth_in_km

    # 假定的P波和S波速度
    p_wave_velocity = 6  # km/sec
    s_wave_velocity = 3.5  # km/sec

    # 使用模型來計算抵達時間
    model = TauPyModel(model = "iasp91")
    model = TauPyModel(model = "ak135")
    arrivals = model.get_travel_times(source_depth_in_km = depth_in_km, distance_in_degree = distance_in_degrees)

    # 尋找P波和S波的抵達時間
    P_arrival_time = None
    S_arrival_time = None
    for arrival in arrivals:
        if arrival.name == 'Pn':
            P_arrival_time = arrival.time
        elif arrival.name == 'Sn':
            S_arrival_time = arrival.time
        elif arrival.name == 'P':
            P_arrival_time = arrival.time
        elif arrival.name == 'S':
            S_arrival_time = arrival.time

    # 如果模型無法提供P波或S波的抵達時間，則使用簡單的估計方法
    if P_arrival_time is None:
        # print('simple estimation')
        P_arrival_time = total_distance_in_km / p_wave_velocity

    if S_arrival_time is None:
        # print('simple estimation')
        S_arrival_time = total_distance_in_km / s_wave_velocity

    # print('Estimated P wave arrival time: ', P_arrival_time, 'seconds')
    # print('Estimated S wave arrival time: ', S_arrival_time, 'seconds')

    P_arrival_sample = P_arrival_time * 100
    S_arrival_sample = S_arrival_time * 100

    P_arrival_sample = int(P_arrival_sample)
    S_arrival_sample = int(S_arrival_sample)

    return P_arrival_sample, S_arrival_sample

In [5]:
def get_trigger_sample(input_Stream, p_start_abs, s_start_abs):

    p_max_abs = p_start_abs
    s_max_abs = s_start_abs

    p_wave_window = 12  # 12 sec
    s_wave_window = 12  # 12 sec

    spacing_window =  1  # 1 sec

    sampling_rate = input_Stream[0].stats.sampling_rate

    # Calculate the composite acceleration
    comp_acc = np.sqrt(input_Stream[0].data**2 + input_Stream[1].data**2 + input_Stream[2].data**2)
    
    # Ensure P-wave window doesn't extend into S-wave window
    if p_start_abs + p_wave_window * sampling_rate >= s_start_abs - spacing_window * sampling_rate:
        p_end_abs = s_start_abs - spacing_window * sampling_rate

        s_end_abs = s_start_abs + s_wave_window * sampling_rate
    else:
        p_end_abs = p_start_abs + p_wave_window * sampling_rate
        s_end_abs = s_start_abs + s_wave_window * sampling_rate

    p_start_abs = int(p_start_abs)
    s_start_abs = int(s_start_abs)
    
    p_end_abs = int(p_end_abs)
    s_end_abs = int(s_end_abs)

    p_window_samples = comp_acc[p_start_abs:p_end_abs]
    s_window_samples = comp_acc[s_start_abs:s_end_abs]
    
    if len(p_window_samples) > 0:
        p_max_sample = np.argmax(p_window_samples)
        p_max_abs = max(p_max_abs, p_start_abs + p_max_sample)
    
    if len(s_window_samples) > 0:
        s_max_sample = np.argmax(s_window_samples)
        s_max_abs = max(s_max_abs, s_start_abs + s_max_sample)

    return p_max_abs, s_max_abs

In [6]:
def process_obspy_stream(input_Stream):

    tr = input_Stream[0]


    fsin = 100.0
    fsout = 40.0
    wlen = 2.0
    alpha = 0.05
    freq = 0.5
    maxshift = 80
    shift_event_r = 0.995
    # shift_event_r = 0.0
    scale_amplitude_r = 0.3
    # scale_amplitude_r = 0.0

    dim = int(wlen * fsout)
    indim = int(fsin * wlen) // 2 


    # Apply Filter to Stream
    input_Stream.detrend(type = "linear")
    input_Stream.taper(alpha)
    input_Stream.filter(type = "highpass", freq = freq)

    # Get the wave arrivals
    p_start_relative, s_start_relative = get_wave_arrival(input_Stream)
    
    trigger_time = tr.stats.sac.b * -1
    trigger_sample = trigger_time * 100
    trigger_sample = int(trigger_sample)

    p_start_abs = trigger_sample + p_start_relative # +  6 * 100
    s_start_abs = trigger_sample + s_start_relative + 12 * 100


    # Get the maximum amplitude of waves
    p_max_abs, s_max_abs = get_trigger_sample(input_Stream, p_start_abs, s_start_abs)

    # Split the data
    data = np.vstack([input_Stream[ii].data for ii in range(len(input_Stream))])
    data1 = shift_event(data, maxshift, shift_event_r, p_max_abs, indim)
    data2 = shift_event(data, maxshift, shift_event_r, s_max_abs, indim)

    # Resample data
    data1 = signal.resample(data1, dim, axis = -1)
    data2 = signal.resample(data2, dim, axis = -1)

    # Scale data amplitude
    if scale_amplitude_r:
        data1 = scale_amplitude(data1, scale_amplitude_r)
        data2 = scale_amplitude(data2, scale_amplitude_r)

    # Transpose data
    data1 = data1.transpose()
    data2 = data2.transpose()

    return data1, data2

In [ ]:
def stream_to_spectrogram_ndarray(input_Stream):
    xyz = ['x', 'y', 'z']
    array_list = []
    for i in range(3):
        trace = input_Stream[i]
        trace_acceleration = trace.data

        # 設置 Spectrogram 參數
        fs = trace.stats.sampling_rate  # 取樣率
        nperseg = 256                   # 每個段的數據點數
        noverlap = nperseg // 2         # 重疊的數據點數

        # Draw Spectrogram
        frequencies, times, Sxx = obspy.signal.spectrogram(trace_acceleration, fs=fs, nperseg=nperseg, noverlap=noverlap)

        plt.pcolormesh(times, frequencies, 10 * np.log10(Sxx), shading = 'auto', cmap = 'gray')
        plt.axis('off')

        # Use BytesIO to save Matplotlib image to ram
        img_stream = BytesIO()
        plt.savefig(img_stream, format = 'png', bbox_inches = 'tight', pad_inches = 0)
        img_stream.seek(0)

        # 使用 OpenCV 讀取並縮放圖片
        img = cv2.imdecode(np.frombuffer(img_stream.read(), dtype=np.uint8), 1)
        img_resized = cv2.resize(img, (150, 100))

        # 將縮放後的圖片轉換為 NumPy 陣列
        img_resized_gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
        img_resized_gray_array = np.asarray(img_resized_gray)

        cv2.imwrite('./Spectrogram_test/' + xyz[i] + '.png', img_resized)

        array_list.append(img_resized_gray_array)

    # 合併成一張彩色圖片 (100, 150, 3)
    # Also, ues RGB instead of BGR. Easier to read
    color_image = np.stack([array_list[2], array_list[0], array_list[1]], axis = -1)

    return color_image

In [7]:
RCEC = ['5AF48', '5AE11', '5ADF9', '5AE28', '5AF0F', '5AE83', '5AEBA', '5AFE5', '5AE1C']
IES = ['5AE21', '5AE99', '5AE73', '5AF8A', '5AFA8', '5AEE6']
TAIPEI = ['Taipei101']

station_list = RCEC + IES

stream_list = []
QSIS_train_data_X = np.array([])
QSIS_train_data_list = []

minimum_pga = 2.0

In [8]:
file_name_part = 'HLX'

all = 0
useable = 0

for datadir in station_list:

    datadir = os.path.join('./data_QSIS_Event', datadir)

    sac_files_X = glob.glob(f'{datadir}/*{file_name_part}*.sac')
    sac_files_Y = [s.replace('X', 'Y') for s in sac_files_X]
    sac_files_Z = [s.replace('X', 'Z') for s in sac_files_X]


    for i in range(len(sac_files_X)):
        stream = Stream()

        stream_x = read(sac_files_X[i])
        stream_x[0].data = stream_x[0].data * 0.01
        stream += stream_x

        stream_y = read(sac_files_Y[i])
        stream_y[0].data = stream_y[0].data * 0.01
        stream += stream_y

        stream_z = read(sac_files_Z[i])
        stream_z[0].data = stream_z[0].data * 0.01
        stream += stream_z
        
        trace_x = stream_x[0]
        trace_y = stream_y[0]
        trace_z = stream_z[0]
        

        data_len_X = len(stream[0].data)
        data_len_Y = len(stream[1].data)
        data_len_Z = len(stream[2].data)
        include_stream = False
        include_stream = data_len_X == data_len_Y and data_len_Y == data_len_Z and data_len_Z == data_len_X
        include_stream = include_stream and data_len_X >= 40000


        if include_stream:
            all = all + 1
            pga_xyz = np.sqrt(trace_x.data**2 + trace_y.data**2 + trace_z.data**2)
            pga_total = max(pga_xyz)
            pga_total = pga_total * 100

            print("Total PGA (gal): ", pga_total)
            if pga_total >= minimum_pga:
                useable = useable + 1

                stream_list.append(stream)

print(useable)
print(all)

Total PGA (gal):  3.9983242750167847
Total PGA (gal):  1.2785506434738636
Total PGA (gal):  1.4058752916753292
Total PGA (gal):  1.5941979363560677
Total PGA (gal):  5.7326339185237885
Total PGA (gal):  1.3646799139678478
Total PGA (gal):  1.2097636237740517
Total PGA (gal):  1.211391855031252
Total PGA (gal):  1.4265851117670536
Total PGA (gal):  1.3370213098824024
Total PGA (gal):  1.2910623103380203
Total PGA (gal):  1.20882298797369
Total PGA (gal):  1.2935636565089226
Total PGA (gal):  1.2279185466468334
Total PGA (gal):  1.276502013206482
Total PGA (gal):  1.2649906799197197
Total PGA (gal):  1.3406062498688698
Total PGA (gal):  1.375001110136509
Total PGA (gal):  1.2150573544204235
Total PGA (gal):  1.3145283795893192
Total PGA (gal):  1.3043510727584362
Total PGA (gal):  1.4431956224143505
Total PGA (gal):  1.229164283722639
Total PGA (gal):  1.324150525033474
Total PGA (gal):  1.4167769812047482
Total PGA (gal):  1.6375711187720299
Total PGA (gal):  1.2676977552473545
Total PG

In [9]:
file_name_part = 'HNX'

all = 0
useable = 0

for datadir in TAIPEI:

    datadir = os.path.join('./data_QSIS_Event', datadir)

    sac_files_X = glob.glob(f'{datadir}/*{file_name_part}*.sac')
    sac_files_Y = [s.replace('X', 'Y') for s in sac_files_X]
    sac_files_Z = [s.replace('X', 'Z') for s in sac_files_X]


    for i in range(len(sac_files_X)):
        stream = Stream()

        stream_x = read(sac_files_X[i])
        stream_x[0].data = stream_x[0].data * 0.01
        stream += stream_x

        stream_y = read(sac_files_Y[i])
        stream_y[0].data = stream_y[0].data * 0.01
        stream += stream_y

        stream_z = read(sac_files_Z[i])
        stream_z[0].data = stream_z[0].data * 0.01
        stream += stream_z
        
        trace_x = stream_x[0]
        trace_y = stream_y[0]
        trace_z = stream_z[0]

        trace_y.trim(trace_x.stats.starttime, trace_x.stats.endtime)
        trace_z.trim(trace_x.stats.starttime, trace_x.stats.endtime)


        data_len_X = len(stream[0].data)
        data_len_Y = len(stream[1].data)
        data_len_Z = len(stream[2].data)
        include_stream = False
        include_stream = data_len_X == data_len_Y and data_len_Y == data_len_Z and data_len_Z == data_len_X
        include_stream = include_stream and data_len_X >= 40000


        if include_stream:
            all = all + 1
            pga_xyz = np.sqrt(trace_x.data**2 + trace_y.data**2 + trace_z.data**2)
            pga_total = max(pga_xyz)
            pga_total = pga_total * 100

            print("Total PGA (gal): ", pga_total)
            if pga_total >= minimum_pga:
                useable = useable + 1

                
                stream_list.append(stream)


print(useable)
print(all)

Total PGA (gal):  2.496848627924919
Total PGA (gal):  2.6209386065602303
Total PGA (gal):  2.5482583791017532
Total PGA (gal):  1.144958008080721
Total PGA (gal):  1.1787768453359604
Total PGA (gal):  1.3647917658090591
Total PGA (gal):  1.9140984863042831
Total PGA (gal):  1.4214866794645786
Total PGA (gal):  1.2163996696472168
Total PGA (gal):  1.1863941326737404
Total PGA (gal):  1.4163991436362267
Total PGA (gal):  1.1460179463028908
Total PGA (gal):  1.227311696857214
Total PGA (gal):  1.9569307565689087
Total PGA (gal):  1.3746388256549835
Total PGA (gal):  1.2988773174583912
Total PGA (gal):  1.1310665868222713
Total PGA (gal):  1.204537134617567
Total PGA (gal):  1.334681548178196
Total PGA (gal):  1.1173943988978863
Total PGA (gal):  1.3345464132726192
Total PGA (gal):  1.2854326516389847
Total PGA (gal):  1.392660103738308
Total PGA (gal):  1.6864437609910965
Total PGA (gal):  1.1319348588585854
Total PGA (gal):  1.065942645072937
Total PGA (gal):  1.4948859810829163
Total PG

In [15]:
for i in range(len(stream_list)):
    # print(stream_list[i])
    p_data, s_data = process_obspy_stream(stream_list[i])
    QSIS_train_data_list.append(p_data)
    QSIS_train_data_list.append(s_data)

In [16]:
QSIS_train_data_X = np.stack(QSIS_train_data_list)
QSIS_train_data_X = np.expand_dims(QSIS_train_data_X, axis = -1)

In [17]:
initial_array = np.array([[1., 0., 0.], [0., 1., 0.]])
QSIS_train_data_Y = np.tile(initial_array, (len(stream_list), 1))

print(QSIS_train_data_X.shape)
print(QSIS_train_data_Y.shape)


(964, 80, 3, 1)
(964, 3)


In [18]:
torch.save({'Xevent': QSIS_train_data_X, 'Yevent': QSIS_train_data_Y}, './Spectrogram_data_pth/Spectrogram_event.pth')

In [19]:
datadir = './Spectrogram_data_pth'

train_data_path = os.path.join(datadir, 'Spectrogram_event.pth')
train_data = torch.load(train_data_path)
Xevent = train_data['Xevent']
Yevent = train_data['Yevent']

print(Xevent.shape)
print(Yevent.shape)

(964, 80, 3, 1)
(964, 3)
